In [1]:
from CountryNet import * 
from bs4 import BeautifulSoup
from random import *
import time
import cPickle
import numpy as np
import pandas as pd

with open("/Users/carterraha/Desktop/artists/lyricsArray.pickle", "rb") as output_file:
    loadtest = cPickle.load(output_file)
with open("/Users/carterraha/Desktop/artists/lyricsArray2.pickle", "rb") as output_file:
    loadtest2 = cPickle.load(output_file)
with open("/Users/carterraha/Desktop/artists/lyricsArray3.pickle", "rb") as output_file:
    loadtest3 = cPickle.load(output_file)
    
lyricsToProcess = loadtest + loadtest2 + loadtest3

In [2]:
processedLyrics = []
for idx in range(len(lyricsToProcess)):
    processedLyrics.append([lyricsToProcess[idx].find("font").findAll(text=True)[0],lyricsToProcess[idx].find("h1").findAll(text=True)[0].strip().split(" lyrics -")[0],str(lyricsToProcess[idx]).split("<br/><br/>\n")[1].strip().split("<br/>\n            <br/>\n")[0].replace("<br/>",'')])
for idx, lyric in enumerate(processedLyrics):
    lyric[2] = lyric[2].split("[Thanks")[0]
    lyric[2] = lyric[2].split("""<script type=""")[0]
    lyric[2] = lyric[2].replace("\n\nChorus:", '')
    lyric[2] = lyric[2].replace("\n\nChorus x2", '')
    lyric[2] = lyric[2].replace("\n\nChorus x3", '')
    lyric[2] = lyric[2].replace("\n\nChorus", '')
    lyric[2] = lyric[2].replace("\n\nCHORUS", '')
    lyric[2] = lyric[2].replace("\n\nREPEAT CHORUS", '')
    lyric[2] = lyric[2].replace("\n\n(Chorus)", '')
    processedLyrics[idx] = lyric
    
#Run twice oddly
for index,lyric in enumerate(processedLyrics):
    if "NMPA" in lyric[2]:
        del processedLyrics[index]
for index,lyric in enumerate(processedLyrics):
    if lyric[1] in ["Nuages", "Kim","The Ballad of Nobeard", "Vous et Moi","Vous Et Moi", "Bon Voyage", "Huckleberry Jam","Down Yonder","Turf's Up","Departure", "American Offline"]:
        del processedLyrics[index]

#Run twice oddly
for index,lyric in enumerate(processedLyrics):
    if "NMPA" in lyric[2]:
        del processedLyrics[index]
for index,lyric in enumerate(processedLyrics):
    if lyric[1] in ["Nuages", "Kim","The Ballad of Nobeard", "Vous et Moi","Vous Et Moi", "Bon Voyage", "Huckleberry Jam","Down Yonder","Turf's Up","Departure", "American Offline"]:
        del processedLyrics[index]

[u'George Strait',
 u'Unwound',
 "Give me a bottle,\nOf your very best,\nCause I've got a problem\nI'm gonna drink off my chest.\nI'm gonna spend the night,\nGettin' down,\nCause that woman that I had\nWrapped around my finger just a come nwound.\n\nThat woman that I had wrapped around my\nFinger just a come unwound,\nShe kicked my out of the house and \nTonight I'm whiskey bound.\nWell I'm gonna be,\nThe drunkest fool in town,\nCause that woman that i had\nWrapped around my finger just a come unwound.\n\nWell she packed my bags,\nAnd opened up the door,\nAnd I got a feelin she didnt want me around no more.\nShe caught me in a lie,\nWhen I was messin around,\nAnd that woman that i had \nWrapped around my finger just a come unwound.\n                    \nThat woman that I had wrapped around my\nFinger just a come unwound,\nShe kicked my out of the house and \nTonight I'm whiskey bound.\nWell I'm gonna be,\nThe drunkest fool in town,\nCause that woman that i had\nWrapped around my finge

In [17]:
trainingData = []
for lyric in processedLyrics[:350]:
    trainingData.append(lyric[2])

In [1]:
import tokenizerPM
from utils import load_data, load_model_parameters_theano, generate_sentences
from gru_theano import *
import numpy as np
import theano as theano
import theano.tensor as T
import time
import operator
from gru_theano import *
import sys

In [101]:
testLyric = processedLyrics[1][2]
wholeSong = "SENTENCE_START " + testLyric.replace("\n", " ").strip() + " SENTENCE_END"
x_train = wholeSong[:-1]
y_train = wholeSong[1:]

In [77]:
class GRUTheano:
    
    def __init__(self, word_dim, hidden_dim=128, bptt_truncate=-1):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Initialize the network parameters
        E = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        U = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (6, hidden_dim, hidden_dim))
        W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (6, hidden_dim, hidden_dim))
        V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        b = np.zeros((6, hidden_dim))
        c = np.zeros(word_dim)
        # Theano: Created shared variables
        self.E = theano.shared(name='E', value=E.astype(theano.config.floatX))
        self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))
        self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))
        self.V = theano.shared(name='V', value=V.astype(theano.config.floatX))
        self.b = theano.shared(name='b', value=b.astype(theano.config.floatX))
        self.c = theano.shared(name='c', value=c.astype(theano.config.floatX))
        # SGD / rmsprop: Initialize parameters
        self.mE = theano.shared(name='mE', value=np.zeros(E.shape).astype(theano.config.floatX))
        self.mU = theano.shared(name='mU', value=np.zeros(U.shape).astype(theano.config.floatX))
        self.mV = theano.shared(name='mV', value=np.zeros(V.shape).astype(theano.config.floatX))
        self.mW = theano.shared(name='mW', value=np.zeros(W.shape).astype(theano.config.floatX))
        self.mb = theano.shared(name='mb', value=np.zeros(b.shape).astype(theano.config.floatX))
        self.mc = theano.shared(name='mc', value=np.zeros(c.shape).astype(theano.config.floatX))
        # We store the Theano graph here
        self.theano = {}
        self.__theano_build__()
    
    def __theano_build__(self):
        E, V, U, W, b, c = self.E, self.V, self.U, self.W, self.b, self.c
        
        x = T.ivector('x')
        y = T.ivector('y')
        
        def forward_prop_step(x_t, s_t1_prev, s_t2_prev):
            # This is how we calculated the hidden state in a simple RNN. No longer!
            # s_t = T.tanh(U[:,x_t] + W.dot(s_t1_prev))
            
            # Word embedding layer
            x_e = E[:,x_t]
            
            # GRU Layer 1
            z_t1 = T.nnet.hard_sigmoid(U[0].dot(x_e) + W[0].dot(s_t1_prev) + b[0])
            r_t1 = T.nnet.hard_sigmoid(U[1].dot(x_e) + W[1].dot(s_t1_prev) + b[1])
            c_t1 = T.tanh(U[2].dot(x_e) + W[2].dot(s_t1_prev * r_t1) + b[2])
            s_t1 = (T.ones_like(z_t1) - z_t1) * c_t1 + z_t1 * s_t1_prev
            
            # GRU Layer 2
            z_t2 = T.nnet.hard_sigmoid(U[3].dot(s_t1) + W[3].dot(s_t2_prev) + b[3])
            r_t2 = T.nnet.hard_sigmoid(U[4].dot(s_t1) + W[4].dot(s_t2_prev) + b[4])
            c_t2 = T.tanh(U[5].dot(s_t1) + W[5].dot(s_t2_prev * r_t2) + b[5])
            s_t2 = (T.ones_like(z_t2) - z_t2) * c_t2 + z_t2 * s_t2_prev
            
            # Final output calculation
            # Theano's softmax returns a matrix with one row, we only need the row
            o_t = T.nnet.softmax(V.dot(s_t2) + c)[0]

            return [o_t, s_t1, s_t2]
        
        [o, s, s2], updates = theano.scan(
            forward_prop_step,
            sequences=x,
            truncate_gradient=self.bptt_truncate,
            outputs_info=[None, 
                          dict(initial=T.zeros(self.hidden_dim)),
                          dict(initial=T.zeros(self.hidden_dim))])
        
        prediction = T.argmax(o, axis=1)
        o_error = T.sum(T.nnet.categorical_crossentropy(o, y))
        
        # Total cost (could add regularization here)
        cost = o_error
        
        # Gradients
        dE = T.grad(cost, E)
        dU = T.grad(cost, U)
        dW = T.grad(cost, W)
        db = T.grad(cost, b)
        dV = T.grad(cost, V)
        dc = T.grad(cost, c)
        
        # Assign functions
        self.predict = theano.function([x], o)
        self.predict_class = theano.function([x], prediction)
        self.ce_error = theano.function([x, y], cost)
        self.bptt = theano.function([x, y], [dE, dU, dW, db, dV, dc])
        
        # SGD parameters
        learning_rate = T.scalar('learning_rate')
        decay = T.scalar('decay')
        
        # rmsprop cache updates
        mE = decay * self.mE + (1 - decay) * dE ** 2
        mU = decay * self.mU + (1 - decay) * dU ** 2
        mW = decay * self.mW + (1 - decay) * dW ** 2
        mV = decay * self.mV + (1 - decay) * dV ** 2
        mb = decay * self.mb + (1 - decay) * db ** 2
        mc = decay * self.mc + (1 - decay) * dc ** 2
        
        self.sgd_step = theano.function(
            [x, y, learning_rate, theano.Param(decay, default=0.9)],
            [], 
            updates=[(E, E - learning_rate * dE / T.sqrt(mE + 1e-6)),
                     (U, U - learning_rate * dU / T.sqrt(mU + 1e-6)),
                     (W, W - learning_rate * dW / T.sqrt(mW + 1e-6)),
                     (V, V - learning_rate * dV / T.sqrt(mV + 1e-6)),
                     (b, b - learning_rate * db / T.sqrt(mb + 1e-6)),
                     (c, c - learning_rate * dc / T.sqrt(mc + 1e-6)),
                     (self.mE, mE),
                     (self.mU, mU),
                     (self.mW, mW),
                     (self.mV, mV),
                     (self.mb, mb),
                     (self.mc, mc)
                    ], allow_input_downcast=True)
        
        
    def calculate_total_loss(self, X, Y):
        return np.sum([self.ce_error(x,y) for x,y in zip(X,Y)])
    
    def calculate_loss(self, X, Y):
        # Divide calculate_loss by the number of words
        num_words = np.sum([len(y) for y in Y])
        return self.calculate_total_loss(X,Y)/float(num_words)

In [87]:
def train_with_sgd(model, X_train, y_train, learning_rate=0.001, nepoch=20, decay=0.9,
    callback_every=10000, callback=None):
    num_examples_seen = 0
    for epoch in range(nepoch):
        # For each training example...
        for i in np.random.permutation(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[0], y_train[0], learning_rate, decay)
            num_examples_seen += 1
            # Optionally do callback
            if (callback and callback_every and num_examples_seen % callback_every == 0):
                callback(model, num_examples_seen)            
    return model


In [104]:
gru_country = GRUTheano(len(index_to_word), 128, 24)

/Users/carterraha/Code/FT_GRU/FT_GRU/lib/python2.7/site-packages/ipykernel_launcher.py:105: UserWarning: The Param class is deprecated. Replace Param(default=N) by theano.In(value=N)


In [102]:
SENTENCE_START_TOKEN = "SENTENCE_START"
SENTENCE_END_TOKEN = "SENTENCE_END"
UNKNOWN_TOKEN = "UNKNOWN_TOKEN"
wholeSong = [nltk.word_tokenize(wholeSong)]
word_freq = nltk.FreqDist(itertools.chain(*wholeSong))
vocab = sorted(word_freq.items(), key=lambda x: (x[1], x[0]), reverse=True)[:len(np.unique(wholeSong))]
sorted_vocab = sorted(vocab, key=operator.itemgetter(1))
index_to_word = ["<MASK/>", UNKNOWN_TOKEN] + [x[0] for x in sorted_vocab]
word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(wholeSong):
    wholeSong[i] = [w if w in word_to_index else UNKNOWN_TOKEN for w in sent]

# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in wholeSong])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in wholeSong])


In [116]:
gru_country2 = train_with_sgd(gru_country, X_train, y_train, .001, 10000, .9)

In [30]:
import nltk
import itertools

In [117]:
wholeSong

[['SENTENCE_START',
  'July',
  'moonlight',
  'shines',
  'Your',
  'pretty',
  'little',
  'head',
  'on',
  'my',
  'shoulder',
  'Pull',
  'over',
  'on',
  'the',
  'side',
  'of',
  'the',
  'road',
  'Oh',
  'my',
  'God',
  ',',
  'you',
  "'re",
  'something',
  'Like',
  'nothing',
  'I',
  "'ve",
  'ever',
  'seen',
  'If',
  'I',
  "'m",
  'asleep',
  'girl',
  ',',
  'let',
  'me',
  'dream',
  'Baby',
  'fall',
  'into',
  'my',
  'kiss',
  'It',
  'should',
  'just',
  'happen',
  'like',
  'this',
  'Trust',
  'it',
  'so',
  'much',
  'that',
  'there',
  "'s",
  'no',
  'one',
  'else',
  'but',
  'us',
  'and',
  'This',
  'moment',
  'that',
  'says',
  'it',
  "'s",
  'so',
  'right',
  "'Cause",
  'that',
  "'s",
  'all',
  'we',
  'have',
  'in',
  'this',
  'life',
  'Drink',
  'up',
  'this',
  'love',
  ',',
  'baby',
  ',',
  'give',
  'it',
  'all',
  'we',
  'got',
  'tonight',
  'Summer',
  'honeysuckle',
  'Leaking',
  'through',
  'a',
  'rolled',
  'dow

In [53]:
testLyric.replace("\n", " ").strip()

"July moonlight shines Your pretty little head on my shoulder Pull over on the side of the road Oh my God, you're something Like nothing I've ever seen If I'm asleep girl, let me dream  Baby fall into my kiss It should just happen like this Trust it so much that there's no one else but us and This moment that says it's so right 'Cause that's all we have in this life Drink up this love, baby, give it all we got tonight  Summer honeysuckle Leaking through a rolled down window We both know when that seat lays back Anything can happen So imagine it'll never end Just close your eyes and you can see that we are where we're meant to be  Baby fall into my kiss It should just happen like this Trust it so much that there's no one else but us and This moment that says it's so right 'Cause that's all we have in this life Baby, drink up this love, give it all we got tonight Give it all we got tonight  Baby fall into my kiss It should just happen like this Trust it so much that there's no one else b

In [60]:
wholeSong

[['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', u'a', 'UNK'],
 ['UNK', 'UNK'],
 ['UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', u'a', 'UNK'],
 ['UNK', 'UNK'],
 ['UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK'],
 ['UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UNK', 'UNK'],
 ['UNK', 'UNK', 'UN

In [90]:
len(index_to_word)

112

In [110]:

def print_sentence(s, index_to_word):
    sentence_str = [index_to_word[x] for x in s[1:-1]]
    print(" ".join(sentence_str))
    sys.stdout.flush()
    
def generate_sentence(model, index_to_word, word_to_index, min_length=5):
    # We start the sentence with the start token
    new_sentence = [word_to_index[SENTENCE_START_TOKEN]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[SENTENCE_END_TOKEN]:
        next_word_probs = model.predict(new_sentence)[-1]
        samples = np.random.multinomial(1, next_word_probs)
        sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
        # Seomtimes we get stuck if the sentence becomes too long, e.g. "........" :(
        # And: We don't want sentences with UNKNOWN_TOKEN's
        if len(new_sentence) > 100 or sampled_word == word_to_index[UNKNOWN_TOKEN]:
            return None
    if len(new_sentence) < min_length:
        return None
    return new_sentence

def generate_sentences(model, n, index_to_word, word_to_index):
    for i in range(n):
        sent = None
        while not sent:
            sent = generate_sentence(model, index_to_word, word_to_index)
        print_sentence(sent, index_to_word)

In [118]:
generate_sentences(gru_country2, 25, index_to_word, word_to_index)

says head Just in in this life life Drink up this love , c'mon , give it all we got tonight Give it all we got tonight
SENTENCE_START Pull be know
Baby up tonight tonight c'mon it all we got tonight Give it all we got tonight
on I in in this life Drink Drink up this love , c'mon , give it all we got tonight Give it all we got tonight
Just got have have in this life Drink up this love , c'mon , give it all we got tonight Give it all we got tonight
into and it all we got tonight
can Oh on fall
That Give all all we got tonight Give it all we got tonight
We I through Give it it all we got tonight
seat have got this life Drink up this love , c'mon , give it all we got tonight Give it all we got tonight
let you we got tonight
but we in in this life life Drink up this love , c'mon , give it all we got tonight Give it all we got tonight
rolled your dream in in this life Drink Drink up this love , c'mon , give it all we got tonight Give it all we got tonight
This ever end Just have in this life

In [100]:
word_to_index

{"'Cause": 74,
 "'ll": 65,
 "'m": 64,
 "'re": 73,
 "'s": 110,
 "'ve": 63,
 ',': 108,
 '<MASK/>': 0,
 'Anything': 62,
 'Baby': 103,
 'Drink': 72,
 'Give': 71,
 'God': 61,
 'I': 70,
 'If': 60,
 'It': 99,
 'Just': 59,
 'Leaking': 58,
 'Like': 57,
 'Oh': 56,
 'Pull': 55,
 'SENTENCE_STARTJuly': 54,
 'So': 53,
 'Summer': 52,
 'That': 51,
 'This': 98,
 'Trust': 97,
 'UNKNOWN_TOKEN': 1,
 'We': 50,
 'Your': 49,
 'a': 48,
 'all': 107,
 'and': 102,
 'are': 47,
 'asleep': 46,
 'baby': 45,
 'back': 44,
 'be': 43,
 'both': 42,
 'but': 96,
 "c'mon": 41,
 'can': 69,
 'close': 40,
 'down': 39,
 'dream': 38,
 'drink': 37,
 'else': 95,
 'end': 36,
 'ever': 35,
 'eyes': 34,
 'fall': 94,
 'girl': 33,
 'give': 93,
 'got': 105,
 'happen': 101,
 'have': 92,
 'head': 32,
 'honeysuckle': 31,
 'imagine': 30,
 'in': 91,
 'into': 90,
 'it': 113,
 'just': 89,
 'kiss': 88,
 'know': 29,
 'lays': 28,
 'let': 27,
 'life': 87,
 'like': 86,
 'little': 26,
 'love': 85,
 'me': 25,
 'meant': 24,
 'moment': 84,
 'moonlight':

In [120]:
processedLyrics[1][2].replace("\n", " ").strip()

"July moonlight shines Your pretty little head on my shoulder Pull over on the side of the road Oh my God, you're something Like nothing I've ever seen If I'm asleep girl, let me dream  Baby fall into my kiss It should just happen like this Trust it so much that there's no one else but us and This moment that says it's so right 'Cause that's all we have in this life Drink up this love, baby, give it all we got tonight  Summer honeysuckle Leaking through a rolled down window We both know when that seat lays back Anything can happen So imagine it'll never end Just close your eyes and you can see that we are where we're meant to be  Baby fall into my kiss It should just happen like this Trust it so much that there's no one else but us and This moment that says it's so right 'Cause that's all we have in this life Baby, drink up this love, give it all we got tonight Give it all we got tonight  Baby fall into my kiss It should just happen like this Trust it so much that there's no one else b

In [125]:
import csv
with open("/Users/carterraha/Desktop/artists/etc/lyricshere2.csv",'w') as f:
    writer = csv.writer(f,delimiter="@")
    for lyric in processedLyrics:
        writer.writerow([lyric[2].replace("\n", " ").strip()])

In [2]:
import numpy as np
import theano as theano
import theano.tensor as T
import time
import operator
from utils import load_data, load_model_parameters_theano, generate_sentences, train_with_sgd
from gru_theano import *
import sys

In [3]:
VOCABULARY_SIZE = 8000
X_train, y_train, word_to_index, index_to_word = load_data("/Users/carterraha/Desktop/artists/etc/lyricshere2.csv", VOCABULARY_SIZE)

Reading CSV file...
Parsed 16292 sentences.
Found 16375 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'tahoe' and appeared 2 times.


In [4]:
# Build your own model (not recommended unless you have a lot of time!)

LEARNING_RATE = 1e-3
NEPOCH = 20
HIDDEN_DIM = 256

model = GRUTheano(VOCABULARY_SIZE, HIDDEN_DIM)

gru_theano.py:112: UserWarning: The Param class is deprecated. Replace Param(default=N) by theano.In(value=N)
  [x, y, learning_rate, theano.Param(decay, default=0.9)],


In [21]:
t1 = time.time()
model.sgd_step(X_train[0], y_train[0], LEARNING_RATE)
t2 = time.time()
print "SGD Step time: ~%f milliseconds" % ((t2 - t1) * 1000.)

SGD Step time: ~586.803913 milliseconds


In [28]:
t1 = time.time()
train_with_sgd(model, X_train, y_train, LEARNING_RATE, 1, decay=0.9)
t2 = time.time()
print "SGD Step time: ~%f milliseconds" % ((t2 - t1) * 1000.)

SGD Step time: ~19985829.493046 milliseconds


In [32]:
generate_sentences(model, 100, index_to_word, word_to_index)

if every this wrong it let it 's like a me
got me there i always
she everywhere living you room and the say she 's girl
about the big out on the summer day us home there at the the a i would do would things that i 'd go in down the road her took off the and he was so but i ai n't ever been for ! ''
keep that 's a realizing one life to a his from a n't got in time .
i 'm that my , it man that 's way .
but i do n't know her some and .
when always 'd winner when we say you is the you there out ( yeah ) , can we wan na go here , us for a palmetto heart love is drive ?
what she left in her before she , when i was and you where it sometimes you were i was to you at home , little can i life that it s only but i her behind you i do through .
i can down at you ’ lonely back , i day you say `` you ca n't gon na look boots that '' like you over now , well that tried to make it baby to love we here for you and you know how to got bar
it i my life i think i out of this mine .
you on my way that nig

nena out of
i house ’ too n't off more than why i no one ’ was i love pouring some road so i ’ now i if you got me a but 've got a about all you was two and it ’ s n't all to but i ’ m you ’ i ’ ll play my hands in the what you have it ’ s , so i will out here like that who so much you good ’ like the way you girl i in her eyes and no
might his `` and your you
here go over on the you why do i the that to not me and my everything there is you ’ boy in an when i ’ m ' never need for you again ’ s stick up over want to you have how everybody two stars up wanted made the stopped my was the me i 've keep from all down my `` will be the light ''
you 're kids her like now she 's not sure me when the way you i said around by do n't i been 'm get me was resistance just i to as the high corvettes 'll you i me that we 'm that you ca n't just be you your on your way you can find the 'd true and got with time who 's your know got your love it 's high 'll dark all ?


In [7]:
def save_model_parameters_theano(model, outfile):
    np.savez(outfile,
        E=model.E.get_value(),
        U=model.U.get_value(),
        W=model.W.get_value(),
        V=model.V.get_value(),
        b=model.b.get_value(),
        c=model.c.get_value())
    print "Saved model parameters to %s." % outfile


In [29]:
save_model_parameters_theano(model, "/Users/carterraha/Desktop/artists/model/params6.npz")

Saved model parameters to /Users/carterraha/Desktop/artists/model/params6.npz.


In [ ]:
model = load_model_parameters_theano('./data/pretrained.npz')

In [ ]:
#Training Epochs completed 
# 1,2 - Oct 18 overnight 
# 3 - Oct 19 AM - took almost the whole day (roughly 10 to 4:30)
# 4 - Oct 19 PM
# 5 - Oct 20 Day
# 6 - Oct 20 overnight
# 7 - Oct 21 overnight